# [실습5] 인공지능을 활용한 이차전지 성능향상 과정에 대한 이해
----

## 실습 목차
---
1. jupyter에서 라이브러리 설치 및 불러오기
1. 리튬이온 양극재 소재의 결정구조 데이터셋 EDA
1. ML을 이용한 결정구조 예측
1. XAI를 이용한 모델 분석
1. AutoML을 이용한 결정구조 예측

## 실습 개요
---
리튬이온 양극재 소재의 결정구조 데이터셋을 기반으로 모델 학습 및 예측정확도를 확인합니다.

---

### 라이브러리 설명
- **pandas**: 데이터프레임을 다루기 위한 라이브러리
- **sklearn**: 머신러닝을 다루기 위한 라이브러리
- **lightgbm**: LightGBM을 다루기 위한 라이브러리
- **warrings**: 경고 메시지를 다루기 위한 라이브러리
- **imblearn**: 불균형한 데이터셋에 대한 샘플링 및 분류 작업을 위한 라이브러리
- **IPython.display**: IPython 환경에서 다양한 형식의 콘텐츠를 표시하고 조작하기 위한 모듈
- **pickle**: 딕셔너리, 리스트, 클래스 등의 자료형을 변환 없이 그대로 파일로 저장하고 이를 불러올 때 사용하는 모듈

## 1. 라이브러리 불러오기
---

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from IPython.display import display
import pickle

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE

import warnings
warnings.simplefilter('ignore')


## 2. 리튬이온 양극재 소재의 결정구조 데이터셋 EDA
---

- https://materialsproject.org/ 에서 다운받은 후 실습을 위해 정제한 데이터
- 양극재로 사용하는 리튬이온 화합물의 결정구조를 분류하기 위한 데이터셋
- 양극재는 배터리의 성능에 가장 크게 영향을 미치는 많은 요인
- 양극재 소재의 결정구조를 예측할 수 있다면 양극재에 따른 배터리의 성능 유추가 가능 (배터리의 성능에 영향을 미치는 요인이 많아 절대적으로 연결될 수는 없지만, 이런 식의 연구가 반복되고 다양해 진다면 충분히 가능할수도)


|컬럼 명 | 의미 |
|----|----|
| Materials Id| materialsproject.org에 명시된 재료의 고유 ID |
| Formula| 재료의 화학식 |
| Spacegroup| 결정체의 기하학적 및 대칭적 특성(국제결정학회에서 정의된 규칙에 따라 분류) |
| Formation Energy (eV)| 화합물이 형성될 때 소비되거나 방출되는 에너지 |
| E Above Hull (eV)| 가장 안정된 형태로 재료가 분해되는 경우의 에너지 |
| Band Gap (eV)| 전자가 하나의 에너지 상태에서 다른 상태로 이동할 때 필요한 최소 에너지 (작을수록 전기가 잘통함) |
| Nsites| 결정의 단위 셀에 있는 원자의 수 |
| Density (gm/cc)| 고체 결정 물질의 밀도 |
| Volume| 물질의 단위 셀 부피 |
| Has Bandstructure| 띠 구 유무  |
| Crystal System| 원자 또는 이온이 배열된 방식(monoclinic, orthorhombic, triclinic 3가지) |

### 2.1 데이터 불러오기

리튬이온 양극재 소재의 결정구조 데이터셋을 불러옵니다.

In [ ]:
import pandas as pd

# "./data/data.csv" 경로에 있는 CSV 파일을 읽어와 데이터프레임(df)에 저장
df = pd.read_csv("./data/data.csv")

# 데이터프레임(df)을 무작위로 섞기 위해 sample 함수를 사용
# frac=1은 전체 데이터를 사용하고, random_state=75는 재현 가능한 무작위 섞기를 위한 시드 설정
df = df.sample(frac=1, random_state=75)

# 변경된 데이터프레임(df)을 출력
df

### 2.2 데이터 전처리

컬럼이름을 데이터분석 시 용이하도록 특수기호, 공백을 제거하고 소문자로 변경합니다.

In [ ]:
print("columns 변경 전")
display(df.columns)

In [ ]:
# 열 이름 변경 전에 "columns 변경 후"를 출력
print("columns 변경 후")

# 열 이름 변경을 위해 lambda 함수와 map 함수를 사용하여 처리
# 열 이름에서 괄호와 그 이후의 문자열을 제거하고 양쪽 공백을 제거하여 열 이름 리스트를 생성
df.columns = list(map(lambda col: col.split("(")[0].strip(), df.columns))

# 열 이름에 있는 공백을 "_"로 대체하고 모든 문자를 소문자로 변환하여 열 이름 리스트를 생성
df.columns = list(map(lambda col: col.replace(" ", "_").lower(), df.columns))

# 변경된 열 이름들을 출력
df.columns

기본적인 정보를 확인합니다.

In [ ]:
df.info()

**[TODO] 중복된 row가 있는지 확인합니다.**

In [ ]:
df.__________().sum()

- 중복된 row는 없는 것으로 확인 됐습니다.


컬럼 별 유니크한 값과 갯수를 확인합니다.

In [ ]:
# 데이터프레임의 열 이름들을 가져와서 cols 변수에 저장
cols = df.columns

# 데이터프레임의 전체 row 수를 계산하여 len_df 변수에 저장
len_df = len(df)

# 각 열에 대해 반복하여 유니크한 값과 해당 열의 정보를 출력
for col in cols:
    # 현재 열(col)에 대해 유니크한 값들을 추출하여 unique 변수에 저장
    unique = df[col].unique()
    
    # 유니크한 값들의 개수를 계산하여 len_unique 변수에 저장
    len_unique = len(unique)
    
    # 해당 열의 유니크한 값의 개수 출력
    print(f"nunique of {col}: {len_unique}")
    
    # 유니크한 값의 개수와 전체 row 수가 같지 않은 경우에만 유니크한 값들을 출력
    if len_unique != len_df:
        display(unique)
    
    print("------------------------------------------------")

- materials_id, volume 컬럼이 전체 row 수와 같습니다. materials_id는 각 row를 구분하는 id이기 때문입니다.
- 다른 컬럼에는 특별히 이상치는 없는 것 같습니다.(예, 숫자 중 숨어있는 문자열)

numeric 형태의 컬럼에 대해 통계를 확인합니다.

In [ ]:
display(df.head(2))
display(df.describe())

object 형태의 컬럼에 대해 통계를 확인합니다.

In [ ]:
# describe 함수를 사용하여 데이터프레임의 요약 통계 정보를 출력한다.
# include 인자를 사용하여 object 타입의 열에 대한 정보만 출력한다.
df.describe(include=["object"])

### 2.3 시각화를 이용한 데이터 분석

crystal_system 컬럼이 가진 각 값들의 비율을 pie 차트로 확인합니다.


In [ ]:
# crystal_system 열의 값들을 세어서 각 값의 빈도를 계산
# autopct: 자동으로 백분율로 변환하는데 소수점 첫째 자리까지 표시
df.crystal_system.value_counts().plot(kind='pie', autopct='%1.1f%%')

# 타이틀 설정
plt.title('Crystal System Distribution')

# 차트 표시
plt.show()

분석 및 학습을 위해 데이터를 변경합니다.   
- numeric 컬럼들은 정규화를 적용
- object 컬럼들은 숫자로 변경

In [ ]:
# 수치형 데이터를 정규화하기 위한 객체 생성
numerical_transformer = StandardScaler()  

# 범주형 데이터를 순서형으로 변환하기 위한 객체 생성
label_transformer = OrdinalEncoder()  

# 정수형과 실수형 열을 선택하여 리스트로 저장
numeric_cols = df.select_dtypes(['int', 'float']).columns.to_list()
# 문자열과 불리언형 열을 선택하여 리스트로 저장  
obj_cols = df.select_dtypes(['object', 'bool']).columns.to_list()  

display(numeric_cols, obj_cols)


In [ ]:
# ColumnTransformer에는 변환기와 열을 지정하는 (이름, 변환기, 열)의 리스트를 전달
# 이름: 해당 변환기를 식별하는 데 사용, 변환기: 해당 열에 적용되는 변환을 정의, 열: 변환을 적용할 열(또는 열의 인덱스)을 지정
ct = ColumnTransformer([('num', numerical_transformer, numeric_cols), ('non_num', label_transformer, obj_cols)]) 

# ct를 사용하여 df를 변환한 결과를 processed에 저장
processed = ct.fit_transform(df)  

new_df = pd.DataFrame(columns=df.columns, data=processed)
new_df.head()

히스토그램을 이용하여 각 컬럼들의 분포를 시각화 합니다.

In [ ]:
new_df.hist(figsize=(12,12), xrot=45)
plt.show()

**[TODO] 히트맵을 통해 상관계수를 확인합니다.**

In [ ]:
plt.figure(figsize=(8,8))
sns.__________(new_df.corr(), annot=True)
plt.show()

pairplot를 통해 상관계수를 확인합니다.   
pairplot: 변수 간의 관계를 시각화하여 탐색적 데이터 분석(EDA)을 수행하는 데 유용합니다. 변수들 간의 상호작용이나 경향성을 빠르게 파악할 수 있으며, 데이터의 특징을 이해하고 모델링을 위한 전처리 작업을 진행하는 데 도움을 줍니다.

In [ ]:
sns.pairplot(new_df.corr(), hue="crystal_system")

formula를 분리하여 새로운 컬럼으로 추가합니다.

In [ ]:
# 입력으로 분자식 리스트를 받는 함수
def get_els(formulas):
    # 분자식에서 원소와 분자를 찾기 위한 정규표현식 패턴
    pattern = r"([A-Z][a-z]?)(\d*)|\((.*?)\)(\d*)"
    # 분자식 리스트를 저장할 리스트
    elements = []
    # 분자식 리스트를 순회하며 각 분자식에서 원소와 분자를 찾아서 딕셔너리 형태로 저장
    for formula in formulas:
        els = {}
        for e in re.findall(pattern, formula):
            symbol, count, molecule, molecule_count = e
            # 원소인 경우 딕셔너리에 추가
            if symbol:
                els[symbol] = int(count) if count else 1
            # 분자인 경우 딕셔너리에 추가
            else:
                els[f"molecule_{molecule}"] = int(molecule_count) if molecule_count else 1
        # 분자식에서 찾은 원소와 분자를 딕셔너리 형태로 저장한 후 리스트에 추가
        elements.append(els)
    # 리스트를 데이터프레임으로 변환하여 반환
    return pd.DataFrame(elements)

cols = df.columns
els_df = pd.DataFrame(get_els(df[cols[1]]))
els_df



In [ ]:
print(f"shape of before concat: {df.shape}")

all_df = pd.concat([new_df, els_df], axis=1)

print(f"shape of after concat: {all_df.shape}")

In [ ]:
all_df.columns

데이터 분석을 위해 crystal_system 값에 따라 df를 구분합니다.

In [ ]:
cs_dfs = [] # 빈 리스트 생성
unique_crystal_system = df.crystal_system.unique().tolist() # df의 crystal_system 열에서 중복을 제거한 값들을 리스트로 만듦
for crystal_system in unique_crystal_system: # crystal_system 값들을 하나씩 꺼내서 반복
    cs_dfs.append(df[df.crystal_system == crystal_system]) # df에서 crystal_system 열이 현재 값과 같은 행들을 추출하여 리스트에 추가함

In [ ]:
def show_hist_box_each_df(cs_dfs, col):
    colors = ["red", "green", "blue"]
    print(col)
    plt.figure(figsize=(10, 7))
    
    # cs_dfs 리스트에서 데이터프레임을 하나씩 꺼내서 반복문 실행
    for i, cs_df in enumerate(cs_dfs):
        # 히스토그램을 그리기 위한 subplot 생성
        plt.subplot(610 + (i * 2 + 1))
        sns.histplot(
            data = cs_df,
            x = col,
            bins = 20,
            color=colors[i]
        )
        # 박스플롯을 그리기 위한 subplot 생성
        plt.subplot(610 + (i * 2 + 2))
        ax = sns.boxplot(
            data = cs_df,
            x = col,
            color=colors[i]
        )
        
        # 각 subplot에 대한 x축 레이블 지정
        plt.xlabel(f"{unique_crystal_system[i]} / len: {len(cs_df[col])}")
    plt.tight_layout()
    plt.show()

dtype이 int 혹은 float인 컬럼들의 통계치를 확인합니다.

In [ ]:
target_col = "crystal_system"

In [ ]:
numeric_cols = df.select_dtypes(["int", "float"]).columns
group_describe_df = df.groupby(by=target_col).describe()
for col in numeric_cols:
    print(col)
    display(group_describe_df[col].T)
    print()

- describe를 이용하면 통계치를 자세히 알 수 있지만 많은 숫자들을 쉽게 확인하기엔 어려운 것 같습니다.

이번엔 시각화를 통해 데이터를 알아보겠습니다.

In [ ]:
for col in numeric_cols:
    show_hist_box_each_df(cs_dfs, col)

dtype이 object인 컬럼들의 통계치를 확인합니다.

In [ ]:
def show_count_each_df(cs_dfs, col):
    colors = ["red", "green", "blue"]
    print(col) # col 출력
    plt.figure(figsize=(15, 6)) # 그래프 크기 설정
    
    for i, cs_df in enumerate(cs_dfs): # cs_dfs에서 데이터프레임과 인덱스를 가져옴
        plt.subplot(311 + i) # 그래프 위치 설정
        sns.countplot( # countplot 그리기
            data = cs_df, # 데이터프레임 설정
            x = col, # x축 설정
            color=colors[i], # 색상 설정
            order=cs_df[col].value_counts().index # x축 순서 설정
        )
        plt.xticks(rotation=90) # x축 라벨 회전
        plt.xlabel(f"{unique_crystal_system[i]}, len: {len(cs_df[col])}") # x축 라벨 설정
    plt.tight_layout() # 그래프 간격 조정
    plt.show() # 그래프 출력

In [ ]:
object_cols = df.select_dtypes(["object"]).columns
for col in object_cols[1:-1]:
    show_count_each_df(cs_dfs, col)

## 3. ML을 이용한 결정구조 예측
---

### 3.1 컬럼 추가에 따른 성능 변화 확인

**[TODO] 학습에 사용하는 컬럼을 하나씩 늘려가며 성능 변화를 확인합니다.**

In [ ]:
def model_tester(X, y, model):
    # 데이터 분할
    X_train, X_test, y_train, y_test = __________(X, y, test_size=0.2, random_state=42)
    
    # 모델 학습
    model.__________(X_train, y_train)

    # 테스트 데이터 예측
    y_pred = model.__________(X_test)

    # 예측 결과 평가
    accuracy = __________(y_test, y_pred)
    return model, accuracy

학습을 위해 데이터를 구분합니다.


In [ ]:
X = all_df.drop(columns = ["materials_id", target_col]).fillna(0)
y = all_df[[target_col]]
X.columns

컬럼을 하나씩 추가하며 성능 변화를 확인하고 직전의 성능 대비 성능 향상, 하락 여부를 출력합니다.

In [ ]:
accs = [0]
for col in X.columns:
    lgbm, acc = model_tester(X.loc[:, :col], y, LGBMClassifier(objective='multiclass', num_leaves=31, num_class=3))
    if acc > accs[-1]:
        print(f"Accuracy: {acc} | {col} | {round(acc - accs[-1], 4)} | +++++")
    elif acc == accs[-1]:
        print(f"Accuracy: {acc} | {col} | {round(acc - accs[-1], 4)} | =====")
    else:
        print(f"Accuracy: {acc} | {col} | {round(acc - accs[-1], 4)} | -----")
    accs.append(acc)
           
           

pickle을 이용하여 모델을 저장하고 불러옵니다.

In [ ]:
# lgbm 모델을 ./lgbm.pkl 파일에 바이너리 형태로 저장합니다.
pickle.dump(lgbm, open("./lgbm.pkl", "wb")) 


# pickle을 이용하여 저장했던 모델을 불러옵니다.
with open('lgbm.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 테스트 데이터 예측
y_pred = loaded_model.predict(X_test)

# 예측 결과 평가
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

**[TODO] feature_importances를 확인합니다.**

In [ ]:
# lgbm 모델의 feature importance를 데이터프레임으로 변환
lgbm_fi_df = pd.DataFrame([lgbm.feature_importances_], columns=X_train.columns).T

# 컬럼명을 "fi"로 변경
lgbm_fi_df.columns=["fi"]

# ff 컬럼을 기준으로 내림차순 정렬
lgbm_fi_df = lgbm_fi_df.__________(by="fi", ascending=False)

# 결과 출력
lgbm_fi_df


feature_importances의 value가 0 이상이었던 컬럼만 사용하여 학습을 진행해봅니다.

In [ ]:
use_cols = lgbm_fi_df[lgbm_fi_df.fi > 0].index.to_list()
print(use_cols)

model_tester(X[use_cols], y, lgbm)

- 모든 컬럼을 사용하였을 때의 정확도와 같은 것을 볼 수 있습니다.
- 추후 여러 실험을 진행할 때 이 점을 참고하여 학습을 더 빠르게 진행하여 모델 학습 시간을 단축할 수 있습니다.

**[TODO] RandomForestClassifier을 사용하여 예측을 진행합니다.**

In [ ]:
rf, acc = model_tester(X.loc[:, :col], y, __________(random_state=75))
print("Accuracy:", acc)


이번엔 rf의 feature importance를 확인해봅니다.

In [ ]:
rf_fi_df = pd.DataFrame([rf.feature_importances_], columns=X_train.columns).T * 100
rf_fi_df.columns=["fi"]
rf_fi_df.sort_values(by="fi", ascending=False)

SMOTE를 통해 오버샘플링을 적용 후 예측을 진행해봅니다.

오버샘플링을 적용한 데이터셋을 만듭니다.

In [ ]:
smote = SMOTE(random_state=42)
# X_train과 y_train을 oversampling
X_resampled, y_resampled = smote.fit_resample(X_train.fillna(0), y_train)

**[TODO] LGBM에 오버샘플링을 적용 후 성능을 테스트합니다.**

In [ ]:
lgbm_smote = __________(objective='multiclass', num_leaves=31, num_class=3)
model, acc = model_tester(X_resampled, y_resampled, lgbm_smote)
print(f"Accuracy: {acc}")

- 오버샘플링 적용 전 정확도인 0.9852941176470589 보다 성능이 저하됐습니다.

- oversampling을 적용할 때 모델 성능이 저하되는 이유는 다음과 같은 요인들이 있을 수 있습니다

- Oversampling을 적용할 때, 주로 데이터를 복제하거나 합성하는 방법을 사용합니다. 이러한 방법은 모델이 훈련 데이터를 더 자세하게 학습할 수 있도록 도와줍니다. 그러나 모델에 적합하지 않은 합성된 데이터가 추가될 수도 있으며, 이는 모델 성능을 저하시킬 수 있습니다.

- Oversampling은 소수 클래스의 샘플을 증가시키는 것이므로, 훈련 데이터가 중복되어 학습될 가능성이 높아집니다. 이는 모델이 훈련 데이터에 과적합되어 실제 데이터에 대한 일반화 성능이 저하될 수 있습니다.

- Oversampling은 소수 클래스에 대한 예측을 더 정확하게 만들 수 있지만, 다수 클래스에 대한 예측의 신뢰도를 감소시킬 수 있습니다. Oversampling으로 생성된 샘플은 기존 데이터와 유사하지만 완전히 동일하지는 않으며, 이로 인해 모델은 다수 클래스의 패턴을 부분적으로 알게 됩니다.



**[TODO] RF에 오버샘플링을 적용 후 성능을 테스트합니다.**

In [ ]:
rf_smote = __________(random_state=75)
model, acc = model_tester(X_resampled, y_resampled, rf_smote)
print(f"Accuracy: {acc}")

- 오버샘플링 적용 전 정확도인 0.8088235294117647 보다 성능이 향상됐습니다.

## 4. 1. XAI를 이용한 모델 분석
---
XAI 관련 라이브러리를 로드합니다.

- **eli5**: permutation importance를 다루기 위한 라이브러리
- **shap**: shap을 다루기 위한 라이브러리
- **lime**: lime을 다루기 위한 라이브러리

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
from lime import lime_tabular
import shap

각 모델별로 feature importance를 확인하고 2개 모델을 통합하여 feature importance를 확인합니다.

`lgbm_fi_df`, `rf_fi_df`에 min-max normalization을 적용합니다.

In [ ]:
lgbm_fi_df = lgbm_fi_df / lgbm_fi_df.sum()
rf_fi_df = rf_fi_df / rf_fi_df.sum()

`lgbm_fi_df`, `rf_fi_df`를 가진 DataFrame을 fi_df에 저장합니다.

In [ ]:
fi_df = pd.concat([lgbm_fi_df, rf_fi_df], axis=1)
fi_df.columns = ["lgbm_fi", "rf_fi"]
fi_df

`fi_df`의 각 컬럼의 sum 값을 오름차순으로 정렬하여 `fi_sum_df`에 저장합니다.

In [ ]:
fi_sum_df = fi_df.T.sum().sort_values()

`plt.barh`를 이용하여 fi_sum_df에서 feature importance 값이 가장 높은 15개를 시각화합니다.

In [ ]:
plt.barh(fi_sum_df.tail(15).index, fi_sum_df.tail(15).values)

- lgbm과 rf를 동시에 참고하여 feature importance를 계산하였을 때 volume이 가장 값이 높게 나왔습니다.



Permutation Importance는 특성의 순열을 무작위로 섞어서 해당 특성의 중요도를 측정하는 기법입니다.

In [ ]:
# PermutationImportance을 호출 시 lgbm, scoring = "accuracy", random_state = 75를 파라미터로 넘겨줍니다.
# 호출 후 fit(X_test, y_test)의 결과를 perm에 저장합니다.
lgbm = LGBMClassifier(objective='multiclass', num_leaves=31, num_class=3)
lgbm.fit(X_train, y_train)
perm = PermutationImportance(lgbm, scoring = "accuracy", random_state = 75).fit(X_test, y_test)

# eli5.show_weights을 호출 시 perm, top = 15, feature_names = X_test.columns.tolist()를 파라미터로 넘겨 시각화합니다.
eli5.show_weights(perm, top = 15, feature_names = X_test.columns.tolist())

- 이번에도 volume가 가장 값이 높게 나왔습니다.



lime_tabular.LimeTabularExplainer를 이용하여 `lime_explainer`를 선언합니다

In [ ]:
lime_explainer = lime_tabular.LimeTabularExplainer(
    # training_data에 X_train.values를 넘깁니다.
    training_data=X_train.values,
    # feature_names에 X_train.columns를 넘깁니다.
    feature_names=X_train.columns,
    # class_names에 [0, 1]를 넘깁니다.
    class_names=[0, 1],
    # mode는 'classification'로 설정합니다.
    mode = 'classification'
)

lime_explainer.explain_instance를 이용하여 `exp`를 선언합니다.

In [ ]:
exp = lime_explainer.explain_instance(
    # data_row에 X_test.iloc[1]를 넘깁니다.
    data_row=X_test.iloc[1], 
    # predict_fn에 model.predict_proba를 넘깁니다.
    predict_fn=lgbm.predict_proba
)

exp.show_in_notebook를 이용하여 결과를 시각화합니다.

In [ ]:
# exp.show_in_notebook 호출 시 show_table=True를 넘깁니다.
exp.show_in_notebook(show_table=True)


shap.Explainer를 이용하여 `shap_explainer`를 선언합니다.

In [ ]:
shap.initjs()

# shap.Explainer 호출 시 trained_model[1][1]를 넘깁니다.
explainer = shap.KernelExplainer(lgbm.predict_proba, X_train, link="logit")
explainer

`shap_explainer`를 이용하여 `shap_values`를 산출합니다.   
nsamples parameter: SHAP 값을 계산할 때 사용되는 샘플의 개수를 제어하는 parameter 입니다. SHAP 값은 해당 샘플들을 기반으로 계산되며, 샘플의 수가 많을수록 SHAP 값의 추정이 더 정확해집니다.

In [ ]:
shap_values = explainer.shap_values(X_test, nsamples=100)

In [ ]:
shap_values = explainer.shap_values(X_test)

force plot은 개별 데이터 포인트의 SHAP 값과 모델의 예측을 시각화하는 방법 중 하나로 각 특성이 해당 예측에 어떻게 기여하는지를 보여줍니다.

먼저 각 데이터마다 feature의 영향력을 살펴보겠습니다.

In [ ]:
X_test.iloc[0,:]

In [ ]:
# explainer.expected_value[0]은 모델 예측값의 기대값입니다.
# shap_values[0][0,:]은 첫 번째 샘플의 SHAP 값입니다.
# X_test.iloc[0,:]은 첫 번째 샘플의 특성값입니다.
shap.force_plot(explainer.expected_value[0], shap_values[0][0,:], X_test.iloc[0,:])

- 기본적으로 base value인 -0.3201로 예측을 시작하여 최종적으로 output value는 -11.76이 됐습니다.
- 크게는 volume가 -16, band_gap이 -0.4281 만큼의 영향을 끼쳤고 그 외 다른 값들이 +, - 영향을 끼쳤습니다.




이번엔 전체 데이터의 영향력을 보겠습니다.

In [ ]:
shap.force_plot(explainer.expected_value[0], shap_values[0], X_test, link="logit")

- 빨간색은 예측에 긍정적인 영향, 파란색은 부정적인 영향을 뜻합니다.
- 값이 증가함에 따라 긍정과 부정적인 영향이 양분화 되는 것이 아닌 반복되는 것을 볼 수 있습니다.

## 5. AutoML을 이용한 결정구조 예측

---

In [ ]:
train_size = int(len(all_df) * 0.8)
train_size

In [ ]:
# pycaret 라이브러리에서 classification 모듈을 import 합니다.
from pycaret.classification import *

# pycaret의 setup 함수를 사용하여 데이터를 전처리하고 모델링에 필요한 환경을 설정합니다.
# all_df에서 materials_id 컬럼을 제외하고, train_size까지의 데이터를 사용합니다.
# 결측치는 0으로 대체합니다.
# target_col은 타겟 변수의 컬럼명입니다.
# session_id는 랜덤 시드값입니다.
# fold는 교차 검증을 위한 fold 수입니다.
setup(all_df.drop(columns="materials_id").iloc[:train_size].fillna(0), target=target_col, session_id=123, fold=5)


models()를 이용하여 학습에 사용할 수 있는 모델들을 확인합니다.

In [ ]:
models()

compare_models()은 교차 검증을 사용하여 다양한 모델을 평가하고 평가 지표에 기반하여 성능이 가장 좋은 모델을 선택합니다.

In [ ]:
# 모델 비교 및 최적 모델 선택
best_model = compare_models()
best_model

- best_model로 선택된 GBC 모델과 파리미터도 알 수 있습니다.



pull()을 이용하여 학습 결과를 확인합니다.

In [ ]:
pull()

- f1 기준으로 보았을 때 gbc, xgboost, dt, lightgbm이 높은 성능을 보여주고 있습니다.


create_model()을 사용하여 모델을 생성합니다. 


In [ ]:
# 최적 모델 학습
model = create_model(best_model)
model

- 5 fold를 적용 및 GBC를 이용하여 학습하였을 때 성능을 자세하게 확인할 수 있습니다.
- 4개의 fold에서 f1이 1이 나왔고 1개의 fold에서만 1 미만의 성능을 기록하여 f1 mean이 저하된 것을 볼 수 있습니다.

evaluate_model을 이용하여 모델에 대해 살펴봅니다.

In [ ]:
evaluate_model(model)

- model에 대하여 자세히 살펴볼 수 있습니다.
- confusion matrix를 확인하였을 때 class 0으로 예측하였으나 실제로는 1이어서 틀린 것 하나를 제외하고는 모두 예측에 성공한 것을 알 수 있습니다. 이는 prediction error 에서도 확인할 수 있습니다.

plot_model을 이용하여 auc에 대해 알아봅니다.

In [ ]:
plot_model(model, plot = 'auc')

plot_model을 이용하여 confusion_matrix에 대해 알아봅니다.

In [ ]:
plot_model(model, plot = 'confusion_matrix')